In [1]:
import tensorflow as tf
import gudhi as gd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.spatial as spatial
import json
from rich import print
from random import choice, sample
from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from gc import collect
from pickle import load
from sklearn.preprocessing import MinMaxScaler

with open("9701_cech_persistence_entropy_012.pickle", mode = "rb") as data:
    save_dict = load(data)
    train_molecules = save_dict["train_molecules"]
    train_data = save_dict["train_data"]
    
    test_molecules = save_dict["test_molecules"]
    test_data = save_dict["test_data"]
    
%matplotlib inline
del save_dict
collect()

4

In [2]:
print(train_data.shape, test_data.shape)

(7761, 9)
(1940, 9)

# Extract descriptors
(this can be done from rdkit or from the descriptor .csv)

In [3]:
df = pd.read_csv("data/Drugbank_some_descriptors.csv")
cntr = 0
for d in df:
    cntr += 1
print(cntr)

126

### Filter descriptors which are unavailable

In [4]:
descriptors = list()

#---- consider only descriptors which have few missing values and which are floating point
for d in df:
    if df[d].isna().sum() < 800 and df[d].dtype == np.float64:
        descriptors.append(d)

#filter_descriptors = filter(lambda d : "OEselma Descriptors" not in d, descriptors)
#descriptors = list(set(descriptors) - set(filter_descriptors))
num_descriptors = len(descriptors)

#---- select only smiles which have these descriptor values
train_smiles = set(train_molecules)
test_smiles = set(test_molecules)

for i,d in enumerate(descriptors):
    avail_rows = df[np.logical_not(df[d].isna())]
    train_smiles = set(avail_rows["SMILES"]).intersection(train_smiles)
    test_smiles = set(avail_rows["SMILES"]).intersection(test_smiles)

train_idxs = np.asarray(sorted([ train_molecules.index(t) for t in train_smiles ]))
test_idxs = np.asarray(sorted([ test_molecules.index(t) for t in test_smiles ]))

train_molecules = np.asarray(train_molecules)
train_molecules = train_molecules[train_idxs]
train_data = train_data[train_idxs]

test_molecules = np.asarray(test_molecules)
test_molecules = test_molecules[test_idxs]
test_data = test_data[test_idxs]
#----
print(descriptors)

[
    'ACD Descriptors;logP',
    'ACD logD-logP;logP',
    'MOE Descriptors;h_ema',
    'MOE Descriptors;h_emd',
    'MOE Descriptors;h_emd_c',
    'MOE Descriptors;h_logp',
    'MOE Descriptors;h_logs',
    'MOE Descriptors;h_log_dbo',
    'MOE Descriptors;h_log_pbo',
    'MOE Descriptors;h_mr',
    'OEselma Descriptors;bromine_count',
    'OEselma Descriptors;carbon_count',
    'OEselma Descriptors;clorine_count',
    'OEselma Descriptors;ertl_tpsa',
    'OEselma Descriptors;fluorine_count',
    'OEselma Descriptors;hba',
    'OEselma Descriptors;hba_lipinski',
    'OEselma Descriptors;hbd',
    'OEselma Descriptors;hbd_lipinski',
    'OEselma Descriptors;halogen_count',
    'OEselma Descriptors;iodine_count',
    'OEselma Descriptors;max_flex_chain_1',
    'OEselma Descriptors;max_flex_chain_2',
    'OEselma Descriptors;max_flex_chain_3',
    'OEselma Descriptors;max_rigid_chain',
    'OEselma Descriptors;mol_volume_2d',
    'OEselma Descriptors;mw',
    'OEselma Descriptors;npsa',
    'OEselma Descriptors;neg_ioniz',
    'OEselma Descriptors;nitrogen_count',
    'OEselma Descriptors;nonpolar_count',
    'OEselma Descriptors;nonpolar_count_per_mw',
    'OEselma Descriptors;rigid_frag_count',
    'OEselma Descriptors;aromatic_ring_count',
    'OEselma Descriptors;atom_count',
    'OEselma Descriptors;bond_count',
    'OEselma Descriptors;rigid_bond_count',
    'OEselma Descriptors;ring_count',
    'OEselma Descriptors;rot_bond_count',
    'OEselma Descriptors;oxygen_count',
    'OEselma Descriptors;psa',
    'OEselma Descriptors;phosphorous_count',
    'OEselma Descriptors;polar_count',
    'OEselma Descriptors;polar_count_per_mw',
    'OEselma Descriptors;pos_ioniz',
    'OEselma Descriptors;silicon_count',
    'OEselma Descriptors;sulphur_count',
    'OEselma Descriptors;tsa',
    'Ertl TPSA;Ertl TPSA',
    'Exact Mass;Exact Mass',
    'Heavy Atom Count;Heavy Atom Count',
    'Molecular Volume (2D);Molecular Volume (2D)',
    'Molecular Weight;Molecular Weight',
    'NPSA;NPSA',
    'PSA;PSA',
    'Ring Count;Ring Count',
    'Rotatable Bond Count;Rotatable Bond Count',
    'AZlogD74;prediction',
    'AZlogD74;sd_prediction',
    'ChromlogD;prediction',
    'ChromlogD;sd_prediction',
    'ALogP',
    'Mol Weight',
    'Molar Refractivity',
    'Polarizability',
    'TPSA',
    'Solubility DD Class;confidence',
    'Solubility DD Class;credibility',
    'SCScore;SCScore',
    'AZlogD74 (NN);nearest_neighbours[]- GT similarity',
    'AZlogD74 (NN);nearest_neighbours[]- GT measured',
    'clogP;clogp',
    'clogP;clogp_error',
    'OEselma Descriptors;part_flex_chain'
]

In [5]:
print(train_data.shape, test_data.shape)

(7205, 9)
(1787, 9)

### Build numpy arrays of descriptors

In [6]:
collect()
train_labels = np.empty((train_data.shape[0], num_descriptors))
test_labels = np.empty((test_data.shape[0], num_descriptors))
train_labels[:] = np.nan
test_labels[:] = np.nan

for i,mol in enumerate(tqdm(train_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    train_labels[i,:] = np.asarray(data_vec)

for i,mol in enumerate(tqdm(test_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    test_labels[i,:] = np.asarray(data_vec)

  0%|          | 0/7205 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

In [7]:
print(train_labels.shape, test_labels.shape)

(7205, 74)
(1787, 74)

In [8]:
print(not np.isnan(train_labels).any(), not np.isnan(test_labels).any())

True True

In [9]:
with np.printoptions(2, suppress = True):
    _idx = choice(list(range(train_labels.shape[0])))
    print(train_labels[_idx])

[ -1.77  -1.77  15.49  17.06   3.55   4.31   7.99  -0.22  -2.38   0.
   0.    10.     0.   224.39   0.    11.    12.    10.    10.     0.
   0.     5.     4.     3.     4.   513.4  318.3  163.52   0.     8.
  -2.    -0.01   6.     0.    22.    21.     7.     0.     9.     4.
 234.04   0.    12.     0.04   5.     0.     0.   398.   224.39 318.18
  22.   513.4  318.3  163.52 234.04   0.     9.    -5.57   1.75  -5.4
   1.59  -4.13 318.33  74.57  30.76 196.7    1.     0.88   3.44   0.48
   2.01  -6.49  59.     8.  ]

# Normalize labels

In [10]:
train_scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_labels)
train_labels = train_scaler.transform(train_labels)

test_scaler = MinMaxScaler(feature_range=(0, 1)).fit(test_labels)
test_labels = test_scaler.transform(test_labels)

In [11]:
with np.printoptions(2, suppress = True):
    print(train_scaler.inverse_transform(train_labels)[_idx])
    print(train_labels[_idx])

[ -1.78  -1.78  15.49  17.06   3.55   4.31   7.99  -0.22  -2.38   0.
   0.    10.     0.   224.39   0.    11.    12.    10.    10.     0.
   0.     5.     4.     3.     4.   513.4  318.3  163.52   0.     8.
  -2.    -0.01   6.     0.    22.    21.     7.     0.     9.     4.
 234.04   0.    12.     0.04   5.     0.     0.   398.   224.39 318.18
  22.   513.4  318.3  163.52 234.04   0.     9.    -5.57   1.75  -5.4
   1.59  -4.13 318.33  74.57  30.76 196.7    1.     0.88   3.44   0.48
   2.01  -6.49  59.     8.  ]

[0.23 0.23 0.28 0.17 0.1  0.09 0.15 0.88 0.21 0.   0.   0.1  0.   0.29
 0.   0.29 0.25 0.32 0.31 0.   0.   0.14 0.12 0.1  0.14 0.15 0.16 0.09
 0.   0.3  0.3  0.44 0.14 0.   0.15 0.14 0.08 0.   0.12 0.09 0.27 0.
 0.26 0.53 0.38 0.   0.   0.16 0.29 0.16 0.15 0.15 0.16 0.09 0.27 0.
 0.12 0.28 0.18 0.22 0.16 0.21 0.16 0.15 0.15 0.22 0.98 0.85 0.61 0.47
 0.58 0.18 1.   0.18]

# Setup OLS model

In [12]:
from sklearn import linear_model

#creating a regression object
reg = linear_model.LinearRegression()

#runnin OLS on your data, assuming that you already have arrays x and y
reg.fit( train_data, train_labels )
reg.score(train_data, train_labels)

0.21731159396225624

In [13]:
reg.coef_.shape

(74, 9)

In [14]:
# Using 'auto'/'sum_over_batch_size' reduction type.
mae = tf.keras.losses.MeanAbsoluteError()
mae(
    test_scaler.inverse_transform(test_labels),
    test_scaler.inverse_transform(reg.predict(test_data))
).numpy()

2022-01-13 17:19:56.960224: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-13 17:19:57.375603: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6912 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


20.665943145751953

In [15]:
predicted = reg.predict(test_data)

In [16]:
test_labels = test_scaler.inverse_transform(test_labels)
predicted = test_scaler.inverse_transform(predicted)

In [17]:
out_dict = dict()
out_dict["Descriptor"] = list()
out_dict["_sigma"] = list()
out_dict["Average"] = list()
out_dict["Test Average"] = test_labels.mean(axis = 0)
out_dict["MAE"] = np.abs(test_labels - predicted).mean(axis = 0)

for i,d in enumerate(descriptors):
    out_dict["Descriptor"].append(d)
    out_dict["Average"].append(df[d].mean(skipna = True)) # population mean for this descriptor
    out_dict["_sigma"].append(df[d].std(skipna = True)) # population mean for this descriptor
    
out_dict["MAE/sigma"] = out_dict["MAE"]/out_dict["_sigma"]

pd.set_option("display.max_rows", None, "display.max_columns", None)
out_df = pd.DataFrame(data = out_dict)
print(out_dict["MAE"].sum()/len(descriptors)) # THIS IS WHAT tf.keras.losses.MeanAbsoluteError() DOES
out_df = out_df.sort_values("MAE/sigma")
out_df

20.66594302944886

,Descriptor,_sigma,Average,Test Average,MAE,MAE/sigma
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.001152,0.010620
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.046804,0.176412
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.237878
62,Mol Weight,301.265463,374.512520,371.636732,102.591485,0.340535
65,TPSA,133.333536,107.810334,104.397905,50.234762,0.376760
64,Polarizability,29.516618,37.783778,37.861987,11.185619,0.378960
63,Molar Refractivity,74.060489,95.559089,95.940680,28.155237,0.380165
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,102.083594,0.390095
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.832488,0.395462
51,Molecular Volume (2D);Molecular Volume (2D),371.986946,592.330629,587.763424,157.475143,0.423335


In [18]:
latex_dict = dict()
latex_dict["Descriptors"]                              = out_dict["Descriptor"]
latex_dict["$\sigma$"]                                 = out_dict["_sigma"]
latex_dict["$\mu$"]                                    = out_dict["Average"]
latex_dict["$\overline{\mathbf{f}_d}$"]                = out_dict["Test Average"]
latex_dict["$\overline{|\mathbf{y}_d-\mathbf{f}_d|}$"] = out_dict["MAE"] 
latex_dict["Score"]                                    = out_dict["MAE/sigma"]
latex_df = pd.DataFrame(data = latex_dict)
latex_df = latex_df.sort_values("Score")
latex_df

,Descriptors,$\sigma$,$\mu$,$\overline{\mathbf{f}_d}$,$\overline{|\mathbf{y}_d-\mathbf{f}_d|}$,Score
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.001152,0.010620
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.046804,0.176412
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.237878
62,Mol Weight,301.265463,374.512520,371.636732,102.591485,0.340535
65,TPSA,133.333536,107.810334,104.397905,50.234762,0.376760
64,Polarizability,29.516618,37.783778,37.861987,11.185619,0.378960
63,Molar Refractivity,74.060489,95.559089,95.940680,28.155237,0.380165
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,102.083594,0.390095
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.832488,0.395462
51,Molecular Volume (2D);Molecular Volume (2D),371.986946,592.330629,587.763424,157.475143,0.423335


In [19]:
latex_string = latex_df.to_latex(index = False, escape = False)
latex_string = latex_string.replace("_", " ")
latex_string = latex_string.replace("OEselma Descriptors;", "OEselma ")
latex_string = latex_string.replace("&\n"," & ")
latex_string = latex_string.replace("\n&"," & ")
latex_string = latex_string.replace("& \n"," & ")
latex_string = latex_string.replace("MOE Descriptors;", "MOE ")

lines = latex_string.split("\n")
for i,l in enumerate(lines):
    if i < 4 or i > len(lines)-4:
        continue
    
    a = l.split("&")
    b = a[1:]
    c = str.title(a[0].split(";")[0])
    d = [ c ] + b
    lines[i] = "&".join(d)
latex_string = "\n".join(lines)

latex_string = latex_string.replace("&", " & ")

for i in range(40):
    latex_string = latex_string.replace("  ", " ")


print(latex_string)

\begin{tabular}{lrrrrr}
\toprule
 Descriptors & $\sigma$ & $\mu$ & $\overline{\mathbf{f} d}$ & $\overline{|\mathbf{y} 
d-\mathbf{f} d|}$ & Score \\
\midrule
 Oeselma Silicon Count & 0.108439 & 0.004239 & 0.000560 & 0.001152 & 0.010620 \\
 Oeselma Bromine Count & 0.265308 & 0.034856 & 0.033016 & 0.046804 & 0.176412 \\
 Oeselma Iodine Count & 0.287843 & 0.028450 & 0.039731 & 0.068471 & 0.237878 \\
 Mol Weight & 301.265463 & 374.512520 & 371.636732 & 102.591485 & 0.340535 \\
 Tpsa & 133.333536 & 107.810334 & 104.397905 & 50.234762 & 0.376760 \\
 Polarizability & 29.516618 & 37.783778 & 37.861987 & 11.185619 & 0.378960 \\
 Molar Refractivity & 74.060489 & 95.559089 & 95.940680 & 28.155237 & 0.380165 \\
 Oeselma Tsa & 261.688962 & 414.138860 & 413.916620 & 102.083594 & 0.390095 \\
 Moe H Ema & 9.691163 & 8.864515 & 8.293012 & 3.832488 & 0.395462 \\
 Molecular Volume (2D) & 371.986946 & 592.330629 & 587.763424 & 157.475143 & 0.423335 \\
 Oeselma Mol Volume 2D & 371.986946 & 592.330629 & 587.763424 & 157.475143 & 0.423335 \\
 Rotatable Bond Count & 7.213711 & 6.681677 & 6.670397 & 3.116832 & 0.432071 \\
 Oeselma Rot Bond Count & 7.213711 & 6.681677 & 6.670397 & 3.116832 & 0.432071 \\
 Molecular Weight & 227.821937 & 373.328639 & 371.636766 & 102.594187 & 0.450326 \\
 Oeselma Mw & 227.821937 & 373.328639 & 371.636766 & 102.594187 & 0.450326 \\
 Exact Mass & 227.548842 & 372.828687 & 371.268124 & 103.113860 & 0.453150 \\
 Moe H Emd & 10.821464 & 12.580101 & 12.460378 & 4.909811 & 0.453710 \\
 Oeselma Atom Count & 15.766603 & 25.429675 & 25.508674 & 7.195343 & 0.456366 \\
 Heavy Atom Count & 15.757548 & 25.426522 & 25.506995 & 7.195667 & 0.456649 \\
 Moe H Logs & 5.975325 & 9.838051 & 9.752189 & 2.784618 & 0.466019 \\
 Oeselma Phosphorous Count & 0.474829 & 0.122751 & 0.123111 & 0.221552 & 0.466593 \\
 Moe H Emd C & 5.020204 & 6.492526 & 6.514657 & 2.362578 & 0.470614 \\
 Oeselma Bond Count & 16.980448 & 26.935092 & 27.180750 & 8.315940 & 0.489736 \\
 Oeselma Part Flex Chain & 4.183248 & 4.829146 & 4.982652 & 2.107297 & 0.503746 \\
 Oeselma Hba Lipinski & 5.725688 & 6.789826 & 6.635702 & 2.890589 & 0.504846 \\
 Oeselma Fluorine Count & 0.933018 & 0.276967 & 0.304421 & 0.471497 & 0.505346 \\
 Oeselma Polar Count & 5.731492 & 6.059067 & 5.834359 & 2.927670 & 0.510804 \\
 Oeselma Psa & 101.197251 & 106.494461 & 102.077127 & 52.759235 & 0.521350 \\
 Psa & 101.197251 & 106.494461 & 102.077127 & 52.759235 & 0.521350 \\
 Ertl Tpsa & 94.357347 & 102.704204 & 99.096475 & 49.971569 & 0.529599 \\
 Oeselma Ertl Tpsa & 94.357347 & 102.704204 & 99.096475 & 49.971569 & 0.529599 \\
 Oeselma Oxygen Count & 4.226225 & 4.206029 & 4.003358 & 2.271479 & 0.537472 \\
 Oeselma Neg Ioniz & 1.108872 & 0.539802 & 0.452714 & 0.603993 & 0.544691 \\
 Azlogd74 & 0.652538 & 0.729417 & 0.712565 & 0.361008 & 0.553237 \\
 Oeselma Carbon Count & 11.377305 & 17.636929 & 17.901511 & 6.298556 & 0.553607 \\
 Moe H Mr & 0.829737 & 0.327136 & 0.322575 & 0.461708 & 0.556451 \\
 Oeselma Max Flex Chain 1 & 2.645862 & 2.943565 & 3.026861 & 1.480748 & 0.559647 \\
 Oeselma Max Flex Chain 2 & 1.948082 & 1.691916 & 1.711807 & 1.115499 & 0.572614 \\
 Oeselma Max Flex Chain 3 & 1.623217 & 1.148954 & 1.130946 & 0.931127 & 0.573630 \\
 Oeselma Rigid Frag Count & 3.579486 & 5.023363 & 4.985450 & 2.076044 & 0.579984 \\
 Oeselma Hba & 4.799591 & 5.812529 & 5.787913 & 2.916762 & 0.607710 \\
 Oeselma Npsa & 200.207940 & 307.642466 & 311.829856 & 121.831882 & 0.608527 \\
 Npsa & 200.207940 & 307.642466 & 311.829856 & 121.831882 & 0.608527 \\
 Oeselma Rigid Bond Count & 10.405612 & 16.273575 & 16.537213 & 6.348085 & 0.610064 \\
 Oeselma Sulphur Count & 0.642512 & 0.289119 & 0.282037 & 0.396247 & 0.616714 \\
 Oeselma Nonpolar Count & 9.445993 & 12.086764 & 12.627308 & 5.949973 & 0.629894 \\
 Oeselma Clorine Count & 0.515279 & 0.168629 & 0.182988 & 0.324874 & 0.630481 \\
 Oeselma Ring Count & 1.811629 & 2.602073 & 2.672076 & 1.157293 & 0.638813 \\
 Ring Count & 1.811629 & 2.602073 & 2.672076 & 1.15